In [1]:
#import the library
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import seaborn as sns
import logging
#import tensorflow as tf
import os
import anndata

import importlib
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
from matplotlib.colors import LinearSegmentedColormap

import pickle as pkl
from matplotlib.colors import LinearSegmentedColormap

import re
from gprofiler import GProfiler
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri

In [36]:
os.chdir('E:/AAA_Labwork/capenterlab_mtb')
adata = sc.read_h5ad("geneformer_mtb_new.h5ad")#("SeuratProject.h5ad")

In [21]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)

In [22]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths('C:\\Users\\16220\\AppData\\Local\\R\\win-library\\4.3'))

# Load all the R libraries we will be using in the notebook
library(scran)
library(Matrix)
library(Seurat)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(ggplot2)
library(plyr)
library(MAST)
library(clusterExperiment)
library(SingleCellExperiment)
library(monocle3)
library(destiny)
library(circlize)

Exception ignored from cffi callback <function _consolewrite_ex at 0x000001EC7B666EE0>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 0: invalid continuation byte
Exception ignored from cffi callback <function _consolewrite_ex at 0x000001EC7B666EE0>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Exception ignored from cffi callback <function _consolewrite_ex at 0x000001EC7B666EE0>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 1: invalid continuation byte
Exception ignored from cffi callback <function _consolewrite_ex at 0x000001EC7B666EE0>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\

### DEG (Wilcoxon since cannot access count data)

In [ ]:
def DE_to_df(_adata, rank_key, _padj_thresh=0.05, _logfc_thresh=1):
    def process_genes(i, direction):
        _log2foldmask = (_adata.uns[rank_key]['logfoldchanges'][i].astype('double') >= _logfc_thresh) if direction == "up" else (_adata.uns[rank_key]['logfoldchanges'][i].astype('double') <= -_logfc_thresh)
        _pvalmask = _adata.uns[rank_key]['pvals_adj'][i].astype('double') <= _padj_thresh
        _additional = pd.DataFrame({
            i: _adata.uns[rank_key]['names'][i].astype('str')[_log2foldmask & _pvalmask],
            'logfoldchanges_'+i: np.abs(_adata.uns[rank_key]['logfoldchanges'][i].astype('double')[_log2foldmask & _pvalmask])
        })
        _ribo_gene_mask = [gene.startswith('RPL') or gene.startswith('RPS') for gene in _additional[i]]
        _mt_gene_mask = [gene.startswith('MT-') for gene in _additional[i]]
        if len(_ribo_gene_mask)>0 and len(_mt_gene_mask)>0:
            _drop_id = np.array(np.array(_ribo_gene_mask) | np.array(_mt_gene_mask))
            _additional = _additional.sort_values(by='logfoldchanges_'+i, ascending=False)
            _additional = _additional.iloc[_drop_id==False, :].reset_index(drop=True)
        return _additional

    _pass_genes_up = pd.DataFrame()
    _pass_genes_down = pd.DataFrame()

    for i in set(_adata.uns[rank_key]['pvals_adj'].dtype.names):

        _pass_genes_up = pd.concat([_pass_genes_up, process_genes(i, "up")[i]], ignore_index=False, axis=1)
        _pass_genes_down = pd.concat([_pass_genes_down, process_genes(i, "down")[i]], ignore_index=False, axis=1)

    return _pass_genes_up, _pass_genes_down

In [ ]:
def get_df_from_rank(adata, rank_key):
        df = pd.DataFrame(index = adata.uns[rank_key]['names']['True'], columns = ['log2fold','pval_adj'])
        df['log2fold'] = adata.uns[rank_key]['logfoldchanges']['True'].astype('double')
        df['pval_adj'] = adata.uns[rank_key]['pvals_adj']['True'].astype('double')
        return df

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'expansion', reference = 'less', key_added = 'more_vs_less', method = 'wilcoxon')

ranking genes


In [ ]:
pass_genes_up = DE_to_df(adata,rank_key = 'more_vs_less')

### Get the at least 1% expressed genes for NicheNet to determine what receptor is expressed

In [14]:
def get_expressed_genes(adata, pct=0.01):
    expressed_genes = {}

    gene_expression = (adata.to_df() > 0).sum(axis=0) / adata.n_obs
    if isinstance(gene_expression, np.matrix) or hasattr(gene_expression, "A1"):
        gene_expression = np.array(gene_expression).flatten()
    expressed_genes = adata.var_names[gene_expression >= pct].tolist()
    
    return expressed_genes

In [15]:
expressed_genes = get_expressed_genes(adata[adata.obs['expansion'] == 'less'])

In [19]:
pd.DataFrame(expressed_genes).to_csv('expressed_genes_in_less_expanded.csv')